In [2]:
from urllib.request import urlopen, urlretrieve
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import time

### mutopia
Scrape mutopia website for mid files in given categories

http://eremita.di.uminho.pt/mutopia/

In [ ]:
instrument = "Guitar"
composer = ""
style = ""

save_dir = "../data/mutopia_guitar"
%mkdir {save_dir}

url0 = 'http://eremita.di.uminho.pt/mutopia/cgibin/make-table.cgi?startat='
url1 = f'&searchingfor=&Composer={composer}&Instrument={instrument}&Style={style}&collection=&id=&solo=&recent=&timelength=&timeunit=&lilyversion=&preview='

songNumber = 0
linkCount = 10

while linkCount > 0:
    url = url0 + str(songNumber) + url1
    html = urlopen(url)
    soup = BeautifulSoup(html.read())
    links = soup.find_all('a')
    linkCount = 0
    for link in links:
        href = link['href']
        if href.find('.mid') >= 0 or href.find('mids.zip') >= 0:
            linkCount = linkCount + 1
            filename = href.split("/")[-1]
            urlretrieve(href, f'{save_dir}/{filename}')
            songNumber += 1
    time.sleep(10.0)

### Songsterr

In [14]:
import requests

artist = "Portishead"
save_dir = "../../../ai/trainingdata/music/songsterr/portishead"
%mkdir {save_dir}

artist_search_url = f'https://www.songsterr.com/a/ra/songs/byartists.xml?artists="{artist}"'
artist_search_response = requests.get(artist_search_url)
root = ElementTree.fromstring(artist_search_response.content)
song_tags = root.findall("./Song")

print(f'Found {len(song_tags)} songs for artist {artist}')

for song_tag in song_tags:
    song_id = song_tag.get("id")
    song_title = song_tag.find("./title").text.replace("/", "-").replace(":", "-")
    revisions_url = f'https://www.songsterr.com/api/meta/{song_id}/revisions'
    revisions_response = requests.get(revisions_url)
    revisions = revisions_response.json()
    if(isinstance(revisions, list) and len(revisions) > 0):
        latest_revision = revisions[0]
        revision_id = latest_revision["revisionId"]
        song_url = latest_revision["source"]
        extension = song_url.split(".")[-1]
        filename = f'{artist} - {song_title} (s{song_id} r{revision_id}).{extension}'
        print(f'Downloading song {filename}')
        urlretrieve(song_url, f'{save_dir}/{filename}')
    


mkdir: ../../../ai/trainingdata/music/songsterr/portishead: File exists
Found 31 songs for artist Portishead
[{'revisionId': 12825, 'songId': 10152, 'description': 'Initial revision', 'createdAt': '2009-01-03T16:56:32.000Z', 'source': 'https://d12drcwhcokzqv.cloudfront.net/3621366.gp4', 'personId': 703, 'person': 'ruvik', 'title': 'Wandering Stars', 'artist': 'Portishead', 'tracksCount': 3, 'reports': [], 'commentsCount': 0}]
[{'revisionId': 597484, 'songId': 10153, 'description': 'Added bars\r\n(via editor)', 'createdAt': '2023-07-24T18:31:24.000Z', 'source': 'https://d12drcwhcokzqv.cloudfront.net/58406991.gp', 'personId': 2560173, 'person': 'Olena', 'title': 'Mysterons', 'artist': 'Portishead', 'tracksCount': 2, 'reports': [], 'commentsCount': 0}, {'revisionId': 12826, 'songId': 10153, 'description': 'Initial revision', 'createdAt': '2009-01-03T17:00:11.000Z', 'source': 'https://d12drcwhcokzqv.cloudfront.net/3621367.gp4', 'personId': 703, 'person': 'ruvik', 'title': 'Mysterons', 'art

KeyError: 0